<a href="https://colab.research.google.com/github/ketanhdoshi/ml/blob/master/lib/debug_lib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
gn_path = 'gdrive/My Drive/Colab Notebooks'  #change dir to your project folder

import sys
sys.path.insert(1, gn_path + '/exp')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
from nb_training import Callback

### Performance Profiling

In [ ]:
! pip install line_profiler
%load_ext line_profiler

In [ ]:
%lprun -f previous_applications previous_applications(full_pa_path)

In [ ]:
%lprun -f _my_roll_cat -f _my_roll_cont my_pa(full_pa_path)

In [ ]:
%lprun -f TabularItemList._roll_cat -f TabularItemList._roll_cont foo_il.time_rollup(df_name='previous_application', index=['SK_ID_CURR'], group_cols=['NAME_CONTRACT_STATUS'], cat_cols=pa_agg_cat, agg_cont = pa_agg_cont)

### Memory Profiling

In [ ]:
import resource
usage = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

In [ ]:
import linecache
import os
import tracemalloc

def display_top(snapshot, key_type='lineno', limit=3):
    snapshot = snapshot.filter_traces((
        tracemalloc.Filter(False, "<frozen importlib._bootstrap>"),
        tracemalloc.Filter(False, "<unknown>"),
    ))
    top_stats = snapshot.statistics(key_type)

    print("Top %s lines" % limit)
    for index, stat in enumerate(top_stats[:limit], 1):
        frame = stat.traceback[0]
        # replace "/path/to/module/file.py" with "module/file.py"
        filename = os.sep.join(frame.filename.split(os.sep)[-2:])
        print("#%s: %s:%s: %.1f KiB"
              % (index, filename, frame.lineno, stat.size / 1024))
        line = linecache.getline(frame.filename, frame.lineno).strip()
        if line:
            print('    %s' % line)

    other = top_stats[limit:]
    if other:
        size = sum(stat.size for stat in other)
        print("%s other: %.1f KiB" % (len(other), size / 1024))
    total = sum(stat.size for stat in top_stats)
    print("Total allocated size: %.1f KiB" % (total / 1024))

In [ ]:
tracemalloc.start()

hcdb = TabularDataBundle(main_file_path, related_csv_paths=related_csv_paths)
train_il = hcdb._do_load(**hcdb.load_params)

snapshot = tracemalloc.take_snapshot()
display_top(snapshot)

current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
tracemalloc.stop()

In [ ]:
!pip install objgraph
import objgraph
objgraph.show_most_common_types()

In [ ]:
x = []
y = [x, [x], dict(x=x)]
objgraph.show_refs([hcdb])

In [ ]:
def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size

In [ ]:
get_size(thc_app.db.train_ds.x.items) / 1024 **2

In [ ]:
sys.getsizeof(thc_app.db.train_ds.x.items) / 1024 **2

In [ ]:
import objgraph
objgraph.show_refs([thc_app])

In [ ]:
objgraph.show_backrefs([thc_app])

In [ ]:
objgraph.show_growth(limit=10)
thc_app = AppTabular()
thc_app.load_data(TabularHomeCreditDataBundle, main_file_path, None, related_csv_paths)
objgraph.show_growth()

In [ ]:
tracemalloc.start()

thc_app = AppTabular()
#thc_app.load_data(TabularHomeCreditDataBundle, main_file_path, test_file_path, related_csv_paths)
thc_app.load_data(TabularHomeCreditDataBundle, main_file_path, None, related_csv_paths)

snapshot = tracemalloc.take_snapshot()
display_top(snapshot)

current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
tracemalloc.stop()

In [ ]:
from guppy import hpy
def dump_heap(h, i):
   “””
   @param h: The heap (from hp = hpy(), h = hp.heap())
   @param i: Identifier str
   “””
  
   print “Dumping stats at: {0}”.format(i)
   print ‘Memory usage: {0} (MB)’.format(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1024)
   print “Most common types:”
   objgraph.show_most_common_types()
  
   print “heap is:”
   print “{0}”.format(h)
  
   by_refs = h.byrcs
   print “by references: {0}”.format(by_refs)
  
   print “More stats for top element..”
   print “By clodo (class or dict owner): {0}”.format(by_refs[0].byclodo)
   print “By size: {0}”.format(by_refs[0].bysize)
   print “By id: {0}”.format(by_refs[0].byid)

### Stack Trace

In [ ]:

import inspect
from inspect import currentframe, getframeinfo
from pandas.core.common import SettingWithCopyError
def istack():
    try:
      pd.options.mode.chained_assignment = 'raise'
    except SettingWithCopyError:
      print('handling..')
      fns = inspect.trace()
      for fn in fns:
        _, filename, lineno, function, code_context, index = fn
        print (f'File: {filename}, Line: {lineno}, Fn: {function}, Ctx: {code_context}, Idx: {index}')

      frameinfo = getframeinfo(currentframe())
      print(frameinfo.function, frameinfo.lineno, frameinfo.code_context)

### Model Computation Graph

Pytorchviz [library](https://github.com/szagoruyko/pytorchviz) and examples. Additional [notes](https://stackoverflow.com/questions/52468956/how-do-i-visualize-a-net-in-pytorch) and how to save image to a file.

Another Option is [HiddenLayer](https://github.com/waleedka/hiddenlayer) library

In [ ]:
#export

#----------------------------------------------------
# Use Pytorchviz and make_dot library to generate model's back propagation
# graph starting from the loss node
#----------------------------------------------------
def show_graph(model, loss, file_path=None):
  !pip install torchviz
  from torchviz import make_dot

  graph = make_dot(loss, params=dict(model.named_parameters()))
  if (file_path is not None):
    graph.render(file_path, format="png")
  return (graph)

#----------------------------------------------------
# Debug Graph Callback to generate dynamic computation graph
#----------------------------------------------------
class DebugGraphCB(Callback):
  def end_tr(self, ctx):
    ctx.graph = show_graph(ctx.model, ctx.loss)

### Export

In [ ]:
!wget https://raw.githubusercontent.com/ketanhdoshi/ml/master/lib/nb_export.py

--2020-05-16 06:19:16--  https://raw.githubusercontent.com/ketanhdoshi/ml/master/lib/nb_export.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1477 (1.4K) [text/plain]
Saving to: ‘nb_export.py’

nb_export.py        100%[===================>]   1.44K  --.-KB/s    in 0s      

2020-05-16 06:19:17 (268 MB/s) - ‘nb_export.py’ saved [1477/1477]



In [ ]:
from nb_export import notebook2scriptSingle
notebook2scriptSingle(gn_path + '/lib/debug_lib.ipynb', gn_path + '/exp')

Converted gdrive/My Drive/Colab Notebooks/lib/debug_lib.ipynb to gdrive/My Drive/Colab Notebooks/exp/nb_debug.py
